<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/Langchain_HuggingFacePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline

model_id = "facebook/bart-large-cnn"
task = "summarization"

llm = HuggingFacePipeline.from_model_id(model_id=model_id, task=task)

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """{input}"""
prompt = PromptTemplate(template=template, input_variables=["input"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = """
The U.S. will be looking to snag their third straight World Cup title — and its fifth overall.

The U.S. women's national team (USWNT) has held the No. 1 spot in FIFA's rankings for years, and is the odds-on favorite to win once again. But this year's tournament is considered fairly wide open, with several teams having a decent shot at the title.

Indeed, the U.S. has had an interesting pattern of late: winning the World Cup, but losing at the Olympics. The USWNT won the World Cup in 2015, then failed to medal at the 2016 Olympics. They won the 2019 World Cup, then took home the bronze at the Tokyo Olympics in 2021.
"""
answer = llm_chain.run(question)

In [ ]:
answer

"The U.S. women's national team is the odds-on favorite to win this year's World Cup. The USWNT won the World Cup in 2015, then failed to medal at the 2016 Olympics. They won the 2019 World Cup, then took home the bronze at the Tokyo Olympics in 2021."

In [ ]:
len(answer)

250

In [ ]:
len(question)

625

In [ ]:
from langchain import PromptTemplate
template = """
你精通多种语言，是专业的翻译官。你负责{src_lang}到{dst_lang}的翻译工作。
"""

prompt = PromptTemplate.from_template(template)
prompt.format(src_lang="英文", dst_lang="中文")

'\n你精通多种语言，是专业的翻译官。你负责英文到中文的翻译工作。\n'

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
system_message_prompt.format(input_language="English", output_language="Japanese")

SystemMessage(content='You are a helpful assistant that translates English to Japanese.', additional_kwargs={})

In [ ]:
system_template="You are a professional translator that translates {src_lang} to {dst_lang}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt.format_prompt(
    src_lang="English",
    dst_lang="Chinese",
    user_input="Did you eat in this morning?"
).to_messages()

[SystemMessage(content='You are a professional translator that translates English to Chinese.', additional_kwargs={}),
 HumanMessage(content='Did you eat in this morning?', additional_kwargs={}, example=False)]

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector


examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # 可选的样本数据
    examples=examples,
    # 提示词模版
    example_prompt=example_prompt,
    # 格式化的样本数据的最大长度，通过get_text_length函数来衡量
    max_length=25,
    # get_text_length: ...
)
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

dynamic_prompt.format(adjective="big")

'Give the antonym of every input\n\nInput: happy\nOutput: sad\n\nInput: tall\nOutput: short\n\nInput: energetic\nOutput: lethargic\n\nInput: sunny\nOutput: gloomy\n\nInput: windy\nOutput: calm\n\nInput: big\nOutput:'